# Read input data

In [ ]:
import re
import pprint
import os

# Define a pattern to match the relevant data
pattern = r'Running (build/\w+) on (\w+)\nNumber of threads: (\d+)\nRuntime: (\d+\.[\d]+)'

# List all files in the 'data_datasets' directory
all_files = os.listdir('data_datasets')

# Filter files that start with 'runtime_'
runtime_files = [f for f in all_files if f.startswith('runtime_')]

runtimes = {}

for runtime_file in runtime_files:
    # Read the text to parse from 'data_datasets/frontiers.log'
    with open(f'data_datasets/{runtime_file}', 'r') as file:
        file_format = file.read()

    # Find all matches in the string
    matches = re.findall(pattern, file_format)

    # Convert matches to a list of dictionaries
    runtimes[runtime_file[len('runtime_'):-4]] = [{'binary': match[0], 'dataset': match[1], 'threads': int(match[2]), 'runtime': match[3]} for match in matches]

pprint.pprint(runtimes)

# Generate plots on runtime and speedups

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Extract data for plotting
datasets = [entry['dataset'] for entry in runtimes['reference']]
reference_runtimes = [float(entry['runtime']) for entry in runtimes['reference']]
bfs_runtimes = [float(entry['runtime']) for entry in runtimes['bfs']]

# Calculate speedups
speedups = [ref / bfs for ref, bfs in zip(reference_runtimes, bfs_runtimes)]

# Plot runtimes
x = np.arange(len(datasets))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, reference_runtimes, width, label='Reference')
rects2 = ax.bar(x + width/2, bfs_runtimes, width, label='BFS')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Datasets')
ax.set_ylabel('Runtimes (s)')
ax.set_title('Runtimes')
ax.set_xticks(x)
ax.set_xticklabels(datasets, rotation=45, ha='right')
ax.legend()

fig.tight_layout()

plt.show()

# Plot speedups
fig, ax = plt.subplots()
rects = ax.bar(x, speedups)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Datasets')
ax.set_ylabel('Speedup (Reference / BFS)')
ax.set_title('Speedups')
ax.set_xticks(x)
ax.set_xticklabels(datasets, rotation=45, ha='right')

fig.tight_layout()

plt.show()